In [1]:
%pip install -e package_calculate_income

Obtaining file:///Users/mateusz/Desktop/final-assignment/package_calculate_income
  Preparing metadata (setup.py) ... done
  Attempting uninstall: package-calculate-income
    Found existing installation: package-calculate-income 0.0.1
    Uninstalling package-calculate-income-0.0.1:
      Successfully uninstalled package-calculate-income-0.0.1
  Running setup.py develop for package-calculate-income
Note: you may need to restart the kernel to use updated packages.


In [2]:
import configparser
import os
import pandas as pd
import numpy as np
import package_calculate_income.calculate_income_jst.jst_income_calc as jst_calc
import package_calculate_income.calculate_income_stat.mean_income_calc as mean_calc
import package_calculate_income.calculate_income_stat.variance_income_calc as var_calc

In [3]:
dir_name = os.path.dirname("script.ipynb")
config = configparser.ConfigParser()
config.read(os.path.join(dir_name, "config.ini"))

['config.ini']

0010 Wpływy z podatku dochodowego od osób fizycznych\
75621 Udziały gmin w podatkach stanowiących dochód budżetu państw\
75622 Udziały powiatów w podatkach stanowiących dochód budżetu państwa\
75623 Udziały województw w podatkach stanowiących dochód budżetu państwa\
75634 Udziały związków metropolitalnych w podatku dochodowym od osób fizycznych\

WK kod województwa\
PK kod powiatu\
GK kod gminy\
GT kod typu gminy (1 miejska, 2 wiejska, 3 miejsko-wiejska)\

### Opodatkowany dochód dla jednostek samorządów terytorialnych z roku 2020 oraz 2019.

In [4]:
df_gm = jst_calc.income_comparison(config["data"]["y1_f1"], config["data"]["y2_f1"])
df_pow = jst_calc.income_comparison(config["data"]["y1_f2"], config["data"]["y2_f2"])
df_mnpp = jst_calc.income_comparison(config["data"]["y1_f3"], config["data"]["y2_f3"])
df_pow_mnpp = pd.concat([df_pow, df_mnpp])
df_metrop = jst_calc.income_comparison(config["data"]["y1_f4"], config["data"]["y2_f4"])
df_woj = jst_calc.income_comparison(config["data"]["y1_f5"], config["data"]["y2_f5"])

In [5]:
%%time
df_gm

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku
kod,,,,,,,,
020101,BOLESŁAWIEC,02,01,01,gmina,1.028386e+08,108377496.0,-5538912.00
020102,BOLESŁAWIEC,02,01,02,gmina,3.505265e+07,35690532.0,-637884.00
020103,GROMADKA,02,01,03,gmina,9.083731e+06,9023523.0,60208.00
020104,NOWOGRODZIEC,02,01,04,gmina,2.428506e+07,24139706.0,145358.00
020105,OSIECZNICA,02,01,05,gmina,1.485653e+07,15390809.0,-534276.00
...,...,...,...,...,...,...,...,...
321801,DOBRA,32,18,01,gmina,5.110458e+06,5225191.0,-114733.50
321802,ŁOBEZ,32,18,02,gmina,2.245806e+07,23644090.0,-1186034.00
321803,RADOWO MAŁE,32,18,03,gmina,3.931197e+06,4357595.5,-426398.25


In [6]:
%%time
df_pow_mnpp

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku
kod,,,,,,,,
020100,BOLESŁAWIECKI,02,01,00,powiat,2.075759e+08,2.144523e+08,-6876400.0
020200,DZIERŻONIOWSKI,02,02,00,powiat,2.080848e+08,2.136905e+08,-5605664.0
020300,GŁOGOWSKI,02,03,00,powiat,2.821103e+08,2.960282e+08,-13917856.0
020400,GÓROWSKI,02,04,00,powiat,5.543528e+07,5.642302e+07,-987748.0
020500,JAWORSKI,02,05,00,powiat,1.019589e+08,1.029109e+08,-952040.0
...,...,...,...,...,...,...,...,...
306300,LESZNO,30,63,00,miasto npp,2.211996e+08,2.250638e+08,-3864224.0
306400,POZNAŃ,30,64,00,miasto npp,2.431833e+09,2.485035e+09,-53201664.0
326100,KOSZALIN,32,61,00,miasto npp,3.392374e+08,3.421914e+08,-2954064.0


In [7]:
%%time
df_metrop

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku
kod,,,,,,,,
240000,GÓRNOŚLĄSKO-ZAGŁĘBIOWSKA\nMETROPOLIA,24,00,00,metropolia,7.270829e+09,7.515800e+09,-244970496.0


In [8]:
%%time
df_woj

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku
kod,,,,,,,,
020000,DOLNOŚLĄSKIE,02,00,00,wojewodztwo,8.986697e+09,9.157574e+09,-170876928.0
040000,KUJAWSKO-POMORSKIE,04,00,00,wojewodztwo,4.871153e+09,5.007150e+09,-135997440.0
060000,LUBELSKIE,06,00,00,wojewodztwo,4.076763e+09,4.201243e+09,-124480512.0
080000,LUBUSKIE,08,00,00,wojewodztwo,2.430983e+09,2.519327e+09,-88344064.0
100000,ŁÓDZKIE,10,00,00,wojewodztwo,6.682968e+09,6.837137e+09,-154169344.0
120000,MAŁOPOLSKIE,12,00,00,wojewodztwo,9.673090e+09,9.759448e+09,-86358016.0
140000,MAZOWIECKIE,14,00,00,wojewodztwo,2.376420e+10,2.437245e+10,-608249856.0
160000,OPOLSKIE,16,00,00,wojewodztwo,2.261440e+09,2.322817e+09,-61377024.0
180000,PODKARPACKIE,18,00,00,wojewodztwo,4.105140e+09,4.196095e+09,-90954752.0


### Średni dochód opodatkowany dla poszczególnych jednostek samorządu terytorialnego.

In [9]:
%%time
df_gm_mean = mean_calc.mean_inc_gm(df_gm, config["data"]["gm_lud"])
df_gm_mean

CPU times: user 245 ms, sys: 21.4 ms, total: 266 ms
Wall time: 326 ms


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku,ludnosc,sredni-opod-doch
kod,,,,,,,,,,
020101,BOLESŁAWIEC,02,01,01,gmina,1.028386e+08,108377496.0,-5538912.00,38486.0,26197.093581
020102,BOLESŁAWIEC,02,01,02,gmina,3.505265e+07,35690532.0,-637884.00,14863.0,23121.401054
020103,GROMADKA,02,01,03,gmina,9.083731e+06,9023523.0,60208.00,5317.0,16749.328346
020104,NOWOGRODZIEC,02,01,04,gmina,2.428506e+07,24139706.0,145358.00,10966.0,21711.550838
020105,OSIECZNICA,02,01,05,gmina,1.485653e+07,15390809.0,-534276.00,7288.0,19985.219120
...,...,...,...,...,...,...,...,...,...,...
321801,DOBRA,32,18,01,gmina,5.110458e+06,5225191.0,-114733.50,2057.0,24357.084090
321802,ŁOBEZ,32,18,02,gmina,2.245806e+07,23644090.0,-1186034.00,3767.0,58448.897553
321803,RADOWO MAŁE,32,18,03,gmina,3.931197e+06,4357595.5,-426398.25,3577.0,10774.712293


In [10]:
%%time
df_pow_mnpp_mean = mean_calc.mean_inc_pow(df_pow, config["data"]["pow_lud"])
df_pow_mnpp_mean

CPU times: user 39 ms, sys: 4.66 ms, total: 43.7 ms
Wall time: 77.5 ms


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku,ludnosc,sredni-opod-doch
kod,,,,,,,,,,
020100,BOLESŁAWIECKI,02,01,00,powiat,207575920.0,214452320.0,-6876400.0,89762.0,22671.708560
020200,DZIERŻONIOWSKI,02,02,00,powiat,208084800.0,213690464.0,-5605664.0,99935.0,20413.737329
020300,GŁOGOWSKI,02,03,00,powiat,282110336.0,296028192.0,-13917856.0,88447.0,31270.561863
020400,GÓROWSKI,02,04,00,powiat,55435276.0,56423024.0,-987748.0,34552.0,15729.424268
020500,JAWORSKI,02,05,00,powiat,101958856.0,102910896.0,-952040.0,49734.0,20098.857826
...,...,...,...,...,...,...,...,...,...,...
321400,STARGARDZKI,32,14,00,powiat,271237664.0,272116096.0,-878432.0,120132.0,22135.589195
321500,SZCZECINECKI,32,15,00,powiat,149047264.0,155109920.0,-6062656.0,77258.0,18913.867961
321600,ŚWIDWIŃSKI,32,16,00,powiat,84305408.0,87697984.0,-3392576.0,46394.0,17815.312538


In [11]:
%%time
df_woj_mean = mean_calc.mean_inc_woj(df_woj, config["data"]["woj_lud"])
df_woj_mean

CPU times: user 13.8 ms, sys: 2.11 ms, total: 15.9 ms
Wall time: 40.3 ms


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku,ludnosc,sredni-opod-doch
kod,,,,,,,,,,
020000,DOLNOŚLĄSKIE,02,00,00,wojewodztwo,8.986697e+09,9.157574e+09,-170876928.0,2891321,30472.185429
040000,KUJAWSKO-POMORSKIE,04,00,00,wojewodztwo,4.871153e+09,5.007150e+09,-135997440.0,2061942,23160.883493
060000,LUBELSKIE,06,00,00,wojewodztwo,4.076763e+09,4.201243e+09,-124480512.0,2095258,19075.580921
080000,LUBUSKIE,08,00,00,wojewodztwo,2.430983e+09,2.519327e+09,-88344064.0,1007145,23664.088402
100000,ŁÓDZKIE,10,00,00,wojewodztwo,6.682968e+09,6.837137e+09,-154169344.0,2437970,26874.528704
120000,MAŁOPOLSKIE,12,00,00,wojewodztwo,9.673090e+09,9.759448e+09,-86358016.0,3410441,27807.024416
140000,MAZOWIECKIE,14,00,00,wojewodztwo,2.376420e+10,2.437245e+10,-608249856.0,5425028,42945.836846
160000,OPOLSKIE,16,00,00,wojewodztwo,2.261440e+09,2.322817e+09,-61377024.0,976774,22698.168043
180000,PODKARPACKIE,18,00,00,wojewodztwo,4.105140e+09,4.196095e+09,-90954752.0,2121229,18973.186197


In [12]:
df_gm_var = var_calc.income_variance_gm(df_gm_mean, df_pow_mnpp_mean)
df_gm_var

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/frame.py:9130: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku,ludnosc,sredni-opod-doch,"(sredni-opod-doch, mean)","(sredni-opod-doch, std)",por_mean
kod,,,,,,,,,,,,,
020100,BOLESŁAWIECKI,02,01,00,powiat,207575920.0,214452320.0,-6876400.0,89762.0,22671.708560,20862.097014,3574.992505,0.086742
020200,DZIERŻONIOWSKI,02,02,00,powiat,208084800.0,213690464.0,-5605664.0,99935.0,20413.737329,28936.073143,21990.175127,0.294523
020300,GŁOGOWSKI,02,03,00,powiat,282110336.0,296028192.0,-13917856.0,88447.0,31270.561863,26400.874422,9381.843282,0.184452
020400,GÓROWSKI,02,04,00,powiat,55435276.0,56423024.0,-987748.0,34552.0,15729.424268,22071.375306,13468.356720,0.287338
020500,JAWORSKI,02,05,00,powiat,101958856.0,102910896.0,-952040.0,49734.0,20098.857826,20515.037313,3822.228204,0.020287
...,...,...,...,...,...,...,...,...,...,...,...,...,...
321400,STARGARDZKI,32,14,00,powiat,271237664.0,272116096.0,-878432.0,120132.0,22135.589195,22996.840423,9125.356659,0.037451
321500,SZCZECINECKI,32,15,00,powiat,149047264.0,155109920.0,-6062656.0,77258.0,18913.867961,21056.078488,7449.109028,0.101738
321600,ŚWIDWIŃSKI,32,16,00,powiat,84305408.0,87697984.0,-3392576.0,46394.0,17815.312538,18089.311365,10193.677741,0.015147


In [13]:
%%time
df_gm_pow_mnpp = pd.concat([df_pow_mnpp_mean, df_gm_mean])
df_pow_var = var_calc.income_variance_pow(df_gm_pow_mnpp, df_woj_mean)
df_pow_var

CPU times: user 11 ms, sys: 2.13 ms, total: 13.2 ms
Wall time: 30 ms


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/frame.py:9130: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku,ludnosc,sredni-opod-doch,"(sredni-opod-doch, mean)","(sredni-opod-doch, std)",por_mean
kod,,,,,,,,,,,,,
020000,DOLNOŚLĄSKIE,02,00,00,wojewodztwo,8.986697e+09,9.157574e+09,-170876928.0,2891321,30472.185429,31018.382555,21009.162048,0.017609
040000,KUJAWSKO-POMORSKIE,04,00,00,wojewodztwo,4.871153e+09,5.007150e+09,-135997440.0,2061942,23160.883493,23361.959476,33794.160456,0.008607
060000,LUBELSKIE,06,00,00,wojewodztwo,4.076763e+09,4.201243e+09,-124480512.0,2095258,19075.580921,15035.633027,11493.931569,0.268692
080000,LUBUSKIE,08,00,00,wojewodztwo,2.430983e+09,2.519327e+09,-88344064.0,1007145,23664.088402,31199.768019,22741.800236,0.241530
100000,ŁÓDZKIE,10,00,00,wojewodztwo,6.682968e+09,6.837137e+09,-154169344.0,2437970,26874.528704,22783.130179,13357.123718,0.179580
120000,MAŁOPOLSKIE,12,00,00,wojewodztwo,9.673090e+09,9.759448e+09,-86358016.0,3410441,27807.024416,24084.458759,17590.435886,0.154563
140000,MAZOWIECKIE,14,00,00,wojewodztwo,2.376420e+10,2.437245e+10,-608249856.0,5425028,42945.836846,26693.821317,25255.416508,0.608831
160000,OPOLSKIE,16,00,00,wojewodztwo,2.261440e+09,2.322817e+09,-61377024.0,976774,22698.168043,28999.357479,18423.062814,0.217287
180000,PODKARPACKIE,18,00,00,wojewodztwo,4.105140e+09,4.196095e+09,-90954752.0,2121229,18973.186197,16878.967060,7907.340901,0.124073


In [14]:
df = [df_gm, df_pow, df_mnpp, df_metrop, df_woj]
df_all_income = pd.concat(df)


with pd.ExcelWriter(config["data"]["results"]) as writer:
    df_all_income.to_excel(writer, sheet_name='doch-jst')
    df_gm_mean.to_excel(writer, sheet_name='gm-srednie-doch')
    df_pow_mnpp_mean.to_excel(writer, sheet_name='pow-mnpp-srednie-doch')
    df_woj_mean.to_excel(writer, sheet_name='woj-srednie-doch')
    df_gm_var.to_excel(writer, sheet_name='pow-stat')
    df_woj_var.to_excel(writer, sheet_name='woj-stat')

FileNotFoundError: [Errno 2] No such file or directory: 'results/results.xlsx'